In [ ]:
import os, re, fastwer
import google.generativeai as genai
from pathlib import Path
from codecarbon import EmissionsTracker

from codecarbon import OfflineEmissionsTracker

config = {
    "save_to_file": True,            # Save emission data to a CSV file
    "log_level": "DEBUG",            # Set logging level to DEBUG for detailed output
    "tracking_mode": "process",      # Track emissions of the current process only
    "output_dir": ".",               # Directory to save the emissions file; '.' denotes the current directory
    "country_iso_code": "ESP",       # ISO code of the country; 'ESP' stands for Spain
    "gpu_ids": [0],                  # List of GPU IDs to monitor; [0] tracks the first GPU
    "allow_multiple_runs": True      # Allow the tracker to be used across multiple runs without overwriting data
}

tracker = OfflineEmissionsTracker(**config)

### UTIL FUNCTIONS #################################################################

def save_text_file(text, output_file):
    if text != "":
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(text)

def verify_text(text, Ref):
    def normalize_text(input_text):
        input_text = re.sub(r'\s*biblioteca\s+nacional\s+de\s+españa\s*$', '', input_text, flags=re.IGNORECASE)
        #input_text = re.sub(r'\s*biblioteca\s+nacional\s+de\s+españa[\s`]*$', '', input_text, flags=re.IGNORECASE)
        input_text = re.sub(r'\s*BIBLIOTECA\s+NACIONAL', '', input_text, flags=re.IGNORECASE)
        lines = input_text.split('\n')
        if lines and len(lines) > 0:
            lines[0] = re.sub(r'\d+', '', lines[0])
            if len(lines) > 1:
                lines[1] = re.sub(r'\d+', '', lines[1])
        text_clean = '\n'.join(lines)
        text_clean = re.sub(r'["„©¶»«—]', '', text_clean)
        text_clean = '\n'.join(line.strip() for line in text_clean.split('\n'))
        text_clean = re.sub(r'\n', ' ', text_clean)
        text_clean = re.sub(r' +', ' ', text_clean)
        text_clean = re.sub(r' +\.', '.', text_clean)
        text_clean = re.sub(r'\s+,', ',', text_clean)
        text_clean = re.sub(r',,', ' ', text_clean)
        pattern = r'(\w+)[\-\u00AD\u2010\u2011]+\s+(\w+)'
        text_clean = re.sub(pattern, r'\1\2', text_clean)
        return text_clean

    normalized_text = normalize_text(text)
    normalized_ref = normalize_text(Ref)
    
    #print("[text]\n", normalized_text)
    #print("[Ref]\n", normalized_ref)
    
    cer = fastwer.score_sent(normalized_text, normalized_ref, char_level=True)
    wer = fastwer.score_sent(normalized_text, normalized_ref)
    print(f'CER:{cer:.2f} - WER:{wer:.2f}')
    return cer, wer

def postprocess(text):
    text = re.sub(r'^\s*```text\s*\n?', '', text)
    text = re.sub(r'```\s*$', '', text)
    return text

### API MODELS #################################################################

gemini_api_key = "XXXXXXXXXXXXXXXX"
genai.configure(api_key=gemini_api_key)
gmodel = genai.GenerativeModel('gemini-2.5-pro-preview-03-25')
#gmodel = genai.GenerativeModel('gemini-2.5-pro-exp-03-25')
#gmodel = genai.GenerativeModel('gemini-2.0-flash-thinking-exp-01-21')
 
def ocr_gemini(filepath):
    text = ""
    try:
        prompt = """
        Perform OCR (Optical Character Recognition) on this image.
        Extract ALL visible text without modernizing or modifying Old Spanish.
        Correct spelling and punctuation while preserving the original language and format.
        Respond ONLY with the extracted text, without additional comments.
        """
        file_content = Path(filepath).read_bytes()
        file_part = {"mime_type": "application/pdf", "data": file_content}
        respuesta = gmodel.generate_content([prompt, file_part])
        text = respuesta.text
    except Exception as e:
        print(f"Error[ocr_gemini]: {str(e)}")
    return text
### PROCESS FLOW #################################################################

def process_directory(directory, directory_ocr, directory_out):
    """ Esta función lee un directorio que contiene imagenes y llama a gemini-2.5-pro-preview-03-25 para extraer el texto. Posteriormente  post procesa el texto obtenido"""
    global check  # Asegurarse de que se use la variable global
    batch = 1000
    count = 1
    for file in sorted(os.scandir(directory), key=lambda x: x.name.lower()):
        try:
            if file.is_file() and file.name.lower().endswith('.pdf'):
                filepath = file.path
                base_name = os.path.splitext(file.name)[0]
                ocr_name = base_name + '.txt'
                file_out = os.path.join(directory_out, ocr_name)
                if not os.path.exists(file_out):
                    print(f"[{count}/{batch}]", filepath)
                    text = ocr_gemini(filepath)
                    text = postprocess(text)
                    save_text_file(text, file_out)
                    count += 1
                    if count >= batch:
                        break
                if verify:
                    ocr_path = os.path.join(directory_ocr, ocr_name)
                    if not os.path.exists(ocr_path):
                        print(f"Error: El archivo {ocr_path} no existe.")
                        continue
                    with open(ocr_path, 'r', encoding='utf-8') as file:
                        Ref = file.read()
                    if not os.path.exists(file_out):
                        print(f"Error: El archivo {file_out} no existe.")
                        continue
                    with open(file_out, 'r', encoding='utf-8') as f:
                        text = f.read()
                    verify_text(text, Ref)
        except Exception as e:
            print(f"Error processing file {filepath}: {str(e)}")
    
directory_in = "./test/pdf/"
directory_ocr = "./dev/test2/"
directory_out = "./test/txt/"
verify = False

tracker.start()
process_directory(directory_in, directory_ocr, directory_out)
emissions = tracker.stop()


[codecarbon INFO @ 11:14:17] offline tracker init
[codecarbon WARNING @ 11:14:17] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 11:14:17] [setup] RAM Tracking...
[codecarbon INFO @ 11:14:17] [setup] CPU Tracking...
[codecarbon DEBUG @ 11:14:17] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 11:14:17] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : [Errno Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : %s] [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:0/energy_uj'
[codecarbon DEBUG @ 11:14:17] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 11:14:17] No CPU tracking mode fo

[1/1000] ./test/pdf/545.pdf
[2/1000] ./test/pdf/546.pdf


[codecarbon INFO @ 11:14:33] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.08745145797729492 W
[codecarbon DEBUG @ 11:14:33] RAM : 0.09 W during 15.01 s [measurement time: 0.0104]
[codecarbon INFO @ 11:14:33] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:14:33] CPU : 42.50 W during 15.02 s [measurement time: 0.0004]
[codecarbon INFO @ 11:14:33] Energy consumed for all GPUs : 0.000060 kWh. Total GPU Power : 14.327974244653587 W
[codecarbon DEBUG @ 11:14:33] GPU : 14.33 W during 15.02 s [measurement time: 0.0081]
[codecarbon INFO @ 11:14:33] 0.000238 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:14:33] last_duration=15.005453579000005
------------------------


[3/1000] ./test/pdf/547.pdf


[codecarbon INFO @ 11:14:48] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08754301071166992 W
[codecarbon DEBUG @ 11:14:48] RAM : 0.09 W during 14.97 s [measurement time: 0.0139]
[codecarbon INFO @ 11:14:48] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:14:48] CPU : 42.50 W during 14.99 s [measurement time: 0.0003]
[codecarbon INFO @ 11:14:48] Energy consumed for all GPUs : 0.000115 kWh. Total GPU Power : 13.256432675428249 W
[codecarbon DEBUG @ 11:14:48] GPU : 13.26 W during 14.99 s [measurement time: 0.0043]
[codecarbon INFO @ 11:14:48] 0.000470 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:14:48] last_duration=14.971972463
------------------------
[codecarbon INFO @ 11:15:03] Energy consumed for RAM : 0.000001 kWh. RAM Power : 0.08763456344604494 W
[codecarbon DEBUG @ 11:15:03] RAM : 0.09 W during 14.97 s [measurement time: 0.0154]
[codecarbon INFO @ 11:15:03] Energy consumed for all CPUs : 0.000531 kWh. T

...

[codecarbon INFO @ 13:55:49] Energy consumed for RAM : 0.000237 kWh. RAM Power : 0.08864164352416992 W
[codecarbon DEBUG @ 13:55:49] RAM : 0.09 W during 14.98 s [measurement time: 0.0138]
[codecarbon INFO @ 13:55:49] Energy consumed for all CPUs : 0.114361 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 13:55:49] CPU : 42.50 W during 15.00 s [measurement time: 0.0003]
[codecarbon INFO @ 13:55:49] Energy consumed for all GPUs : 0.037695 kWh. Total GPU Power : 16.218956146896815 W
[codecarbon DEBUG @ 13:55:49] GPU : 16.22 W during 15.00 s [measurement time: 0.0047]
[codecarbon INFO @ 13:55:49] 0.152293 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 13:55:49] last_duration=14.97625959600009
------------------------
[codecarbon INFO @ 13:56:04] Energy consumed for RAM : 0.000237 kWh. RAM Power : 0.08864164352416992 W
[codecarbon DEBUG @ 13:56:04] RAM : 0.09 W during 14.97 s [measurement time: 0.0146]
[codecarbon INFO @ 13:56:04] Energy consumed for all CPUs : 0.114538 k

[503/1000] ./test/pdf/998.pdf


[codecarbon INFO @ 13:56:19] Energy consumed for RAM : 0.000238 kWh. RAM Power : 0.08864164352416992 W
[codecarbon DEBUG @ 13:56:19] RAM : 0.09 W during 14.97 s [measurement time: 0.0137]
[codecarbon INFO @ 13:56:19] Energy consumed for all CPUs : 0.114715 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 13:56:19] CPU : 42.50 W during 14.99 s [measurement time: 0.0002]
[codecarbon INFO @ 13:56:19] Energy consumed for all GPUs : 0.037835 kWh. Total GPU Power : 16.620738525369642 W
[codecarbon DEBUG @ 13:56:19] GPU : 16.62 W during 14.99 s [measurement time: 0.0045]
[codecarbon INFO @ 13:56:19] 0.152788 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 13:56:19] We apply an energy mix of 174 g.CO2eq/kWh for Spain
[codecarbon DEBUG @ 13:56:19] EmissionsData(timestamp='2025-04-13T13:56:19', project_name='codecarbon', run_id='868e486c-ef2f-47f1-8b33-54ddf9c190fd', experiment_id='5b0fa12a-3dd7-45bb-9766-cc326314d9f1', duration=9720.558401774, emissions=0.026592752937912533, e

[504/1000] ./test/pdf/999.pdf


[codecarbon INFO @ 13:56:34] Energy consumed for RAM : 0.000238 kWh. RAM Power : 0.08864164352416992 W
[codecarbon DEBUG @ 13:56:34] RAM : 0.09 W during 14.97 s [measurement time: 0.0138]
[codecarbon INFO @ 13:56:34] Energy consumed for all CPUs : 0.114892 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 13:56:34] CPU : 42.50 W during 15.00 s [measurement time: 0.0005]
[codecarbon INFO @ 13:56:34] Energy consumed for all GPUs : 0.037904 kWh. Total GPU Power : 16.503001916881463 W
[codecarbon DEBUG @ 13:56:34] GPU : 16.50 W during 15.00 s [measurement time: 0.0067]
[codecarbon INFO @ 13:56:34] 0.153034 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 13:56:34] last_duration=14.974911097000586
------------------------
[codecarbon INFO @ 13:56:41] Energy consumed for RAM : 0.000238 kWh. RAM Power : 0.08864164352416992 W
[codecarbon DEBUG @ 13:56:41] RAM : 0.09 W during 7.81 s [measurement time: 0.0128]
[codecarbon INFO @ 13:56:41] Energy consumed for all CPUs : 0.114985 k